# Proyecto 3 Hiperparametrización con Optuna en modelos de aprendizaje profundo

## 1. Introducción a optuna y configuración inicial:

a. Estudiar los fundamentos y características de Optuna, incluyendo su arquitectura y
métodos de optimización. \\
b. Preparar un entorno de desarrollo en PyTorch y configurar Optuna para integrarse con
modelos de aprendizaje profundo.

------------

### Introducción

La búsqueda de hiperparámetros forma parte de casi todos los proyectos de aprendizaje automático y aprendizaje profundo. Cuando seleccionamos un modelo candidato, nos aseguramos de que generalize a los datos de prueba de la mejor manera posible.

Seleccionar manualmente los mejores hiperparámetros es fácil si se trata de un modelo sencillo como la regresión lineal. Para modelos complejos como las redes neuronales, el ajuste manual es difícil.

Por ejemplo, si entrenamos una red neuronal con sólo capas lineales, aquí tenemos un conjunto potencial de hiperparámetros:

- Número de capas
- Unidades por capa
- Función de activación
- Tasa de aprendizaje
- etc.

A menudo, para optimizar los hiperparámetros se utilizan métodos Grid Search y Random Search.

Por ejemplo, digamos que tenemos 3 valores candidatos para cada una de esas 4 variables, acabamos con 3^4 = 81 experimentos. Para redes más grandes y más valores candidatos, este número se vuelve abrumador.

Estos dos enfoques consumen mucho tiempo y recursos. Los algoritmos de aprendizaje profundo actuales a menudo contienen muchos hiperparámetros, y se tarda días, semanas en entrenar un buen modelo. Simplemente no es posible forzar сombinaciones de hiperparámetros y entrenar modelos separados para cada uno sin ninguna optimización.

### Optuna

Para esto se creo **Optuna**, que es una biblioteca de Python utilizada para la optimización de hiperparámetros.

**Optuna combina mecanismos de muestreo (sampling) y poda (pruning) para proporcionar una optimización eficiente de los hiperparámetros.**

Optuna utiliza el muestreo para explorar el espacio de búsqueda de hiperparámetros. Sugiere nuevos valores de hiperparámetros basándose en ensayos anteriores y en el algoritmo de optimización utilizado. Ofrece distintas estrategias de muestreo:

- Grid Search implementado en [GridSampler](https://optuna.readthedocs.io/en/stable/reference/samplers/generated/optuna.samplers.GridSampler.html#optuna.samplers.GridSampler)
- Random Search implementado en [RandomSampler](https://optuna.readthedocs.io/en/stable/reference/samplers/generated/optuna.samplers.RandomSampler.html#optuna.samplers.RandomSampler)
- Tree-structured Parzen Estimator algorithm implementado en [TPESampler](https://optuna.readthedocs.io/en/stable/reference/samplers/generated/optuna.samplers.TPESampler.html#optuna.samplers.TPESampler)
- CMA-ES based algorithm implementado en [CmaEsSampler](https://optuna.readthedocs.io/en/stable/reference/samplers/generated/optuna.samplers.CmaEsSampler.html#optuna.samplers.CmaEsSampler)
- Algoritmo para activar parámetros fijos parciales implementado en [PartialFixedSampler](https://optuna.readthedocs.io/en/stable/reference/samplers/generated/optuna.samplers.PartialFixedSampler.html#optuna.samplers.PartialFixedSampler)
- Non-dominated Sorting Genetic Algorithm implementado en [NSGAIISampler](https://optuna.readthedocs.io/en/stable/reference/samplers/generated/optuna.samplers.NSGAIISampler.html#optuna.samplers.NSGAIISampler)
- uasi Monte Carlo sampling algorithm implementado en [QMCSampler](https://optuna.readthedocs.io/en/stable/reference/samplers/generated/optuna.samplers.QMCSampler.html#optuna.samplers.QMCSampler)


Tambien proporciona mecanismos para detener y podar tempranamente ensayos poco prometedores. Supervisa continuamente el progreso de las pruebas y elimina aquellas que probablemente no produzcan mejores resultados, ahorrando tiempo y recursos computacionales. Las decisiones de poda se toman con base en los resultados intermedios informados por la función objetivo durante la evaluación de un ensayo.

### Características principales de Optuna:

Según los [autores de Optuna](https://optuna.org/), son tres las características que la hacen destacar:
- Eager search spaces: Automated search for optimal hyperparameters using Python conditionals, loops, and syntax
- State-of-the-art algorithms: Efficiently search large spaces and prune unpromising trials for faster results
- Easy parallelization: Parallelize hyperparameter searches over multiple threads or processes without modifying code

### Flujo de trabajo

El flujo de trabajo de Optuna se resuelve en torno a dos términos:

1. Ensayo (Trial): Una única llamada a una función objetivo.
2. Estudio (Study): Optimización de hiperparámetros basada en una función objetivo. Un estudio tiene como objetivo determinar el conjunto ideal de valores de hiperparámetros mediante la realización de varios ensayos.

### Integración de Optuna y PyTorch

Ahora, vamos a desglosar el proceso de optimización de hiperparámetros con Optuna.

In [1]:
!pip install optuna
import optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 9.0 MB/s eta 0:00:00


In [2]:
import os
import time
import copy

import torch
import torch.nn as nn
import torch.optim as optim

import torchvision
from torchvision import datasets, models, transforms

import numpy as np
import matplotlib.pyplot as plt

## 2. Optimización de un modelo de clasificación de imágenes:

a. Implementar un modelo convencional como ResNet o VGG en PyTorch. \\
b. Utilizar Optuna para optimizar hiperparámetros como tasa de aprendizaje, tamaño de lote, y configuraciones específicas de capas. \\
c. Evaluar las mejoras en precisión y tiempo de entrenamiento tras la optimización de hiperparámetros

El modelo SimpleVGG define capas convolucionales seguidas de capas de pooling y capas completamente conectadas para la clasificación. La función objetivo (objective) carga un subconjunto del conjunto de datos CIFAR10, define hiperparámetros a optimizar con Optuna, entrena el modelo y evalúa su precisión en un conjunto de prueba más pequeño. Optuna se utiliza para encontrar los mejores hiperparámetros en un número limitado de pruebas (n_trials).

In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import optuna

# Modelo VGG
class SimpleVGG(nn.Module):
    def __init__(self, num_classes=10):
        super(SimpleVGG, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.classifier = nn.Sequential(
            nn.Linear(128 * 4 * 4, 256),
            nn.ReLU(inplace=True),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

# Definimos la función objetivo para optimizar con Optuna
def objective(trial):
    # Cargamos un subconjunto del dataset CIFAR10
    transform = transforms.ToTensor()
    cifar10_train = datasets.CIFAR10(root='./data', train=True, download=False, transform=transform) # download=True
    cifar10_test = datasets.CIFAR10(root='./data', train=False, download=False, transform=transform)

    # Usamos un subconjunto más pequeño del dataset
    small_train, _ = torch.utils.data.random_split(cifar10_train, [2000, len(cifar10_train) - 2000])
    small_test, _ = torch.utils.data.random_split(cifar10_test, [500, len(cifar10_test) - 500])

    trainloader = torch.utils.data.DataLoader(small_train, batch_size=64, shuffle=True)
    testloader = torch.utils.data.DataLoader(small_test, batch_size=64, shuffle=False)

    # Definimos los hiperparámetros a optimizar
    lr = trial.suggest_float("lr", 1e-4, 1e-1, log=True)
    weight_decay = trial.suggest_float("weight_decay", 1e-5, 1e-1, log=True)
    momentum = trial.suggest_float("momentum", 0.0, 1.0)

    # Definimos el modelo, el optimizador y la función de pérdida
    model = SimpleVGG(num_classes=10)
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay)
    loss_fn = nn.CrossEntropyLoss()

    # Entrenamos el modelo por una época
    model.train()
    for data, target in trainloader:
        optimizer.zero_grad()
        output = model(data)
        loss = loss_fn(output, target)
        loss.backward()
        optimizer.step()

    # Evaluamos el modelo en el conjunto de prueba
    model.eval()
    correct = 0
    with torch.no_grad():
        for data, target in testloader:
            output = model(data)
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    # Calculamos la precisión en el conjunto de prueba y devolverla como el valor objetivo para Optuna
    accuracy = correct / len(testloader.dataset)
    return accuracy

# Ejecutamos el estudio de Optuna para optimizar los hiperparámetros
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=10)  # Número reducido de pruebas por simplicidad

# Imprimimos los mejores hiperparámetros y el mejor valor objetivo encontrado por Optuna
print("Mejores hiperparámetros: {}".format(study.best_params))
print("Mejor valor objetivo: {}".format(study.best_value))


[I 2024-06-13 10:51:10,287] A new study created in memory with name: no-name-34effd85-739a-4e19-ae0c-b85d8e07a2d4
[I 2024-06-13 10:51:16,071] Trial 0 finished with value: 0.09 and parameters: {'lr': 0.00016146071595973303, 'weight_decay': 0.006100654238471693, 'momentum': 0.2814455422272276}. Best is trial 0 with value: 0.09.
[I 2024-06-13 10:51:22,767] Trial 1 finished with value: 0.1 and parameters: {'lr': 0.07460826413602062, 'weight_decay': 0.05834251494223821, 'momentum': 0.3623394300518419}. Best is trial 1 with value: 0.1.
[I 2024-06-13 10:51:28,325] Trial 2 finished with value: 0.11 and parameters: {'lr': 0.00415360423593682, 'weight_decay': 0.0019891326436610036, 'momentum': 0.3962940986147029}. Best is trial 2 with value: 0.11.
[I 2024-06-13 10:51:34,878] Trial 3 finished with value: 0.09 and parameters: {'lr': 0.0011597352045685033, 'weight_decay': 0.024433094309962874, 'momentum': 0.7780665163047523}. Best is trial 2 with value: 0.11.
[I 2024-06-13 10:51:40,421] Trial 4 fin

Mejores hiperparámetros: {'lr': 0.07110651098699147, 'weight_decay': 8.618140561314041e-05, 'momentum': 0.7779833413083852}
Mejor valor objetivo: 0.164


Los resultados que estamos obteniendo son un poco bajos para el conjunto de datos CIFAR-10, aunque se debe considerar que hemos reducido significativamente la complejidad del modelo y el tamaño del subconjunto de datos.

Ahora modificaremos el codigo un poco, el cual incrementa el número de épocas y usa un subconjunto de datos más grande:

In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import optuna

# Definimos un modelo VGG simplificado
class SimpleVGG(nn.Module):
    def __init__(self, num_classes=10):
        super(SimpleVGG, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.classifier = nn.Sequential(
            nn.Linear(128 * 4 * 4, 256),
            nn.ReLU(inplace=True),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

# Definimos la función objetivo para optimizar con Optuna
def objective(trial):
    # Cargamos un subconjunto del dataset CIFAR10
    transform = transforms.ToTensor()
    cifar10_train = datasets.CIFAR10(root='./data', train=True, download=False, transform=transform)
    cifar10_test = datasets.CIFAR10(root='./data', train=False, download=False, transform=transform)

    # Usamos un subconjunto más pequeño del dataset
    small_train, _ = torch.utils.data.random_split(cifar10_train, [5000, len(cifar10_train) - 5000])
    small_test, _ = torch.utils.data.random_split(cifar10_test, [1000, len(cifar10_test) - 1000])

    trainloader = torch.utils.data.DataLoader(small_train, batch_size=64, shuffle=True)
    testloader = torch.utils.data.DataLoader(small_test, batch_size=64, shuffle=False)

    # Definimos los hiperparámetros a optimizar
    lr = trial.suggest_float("lr", 1e-4, 1e-1, log=True)
    weight_decay = trial.suggest_float("weight_decay", 1e-5, 1e-1, log=True)
    momentum = trial.suggest_float("momentum", 0.0, 1.0)

    # Definimos el modelo, el optimizador y la función de pérdida
    model = SimpleVGG(num_classes=10)
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay)
    loss_fn = nn.CrossEntropyLoss()

    # Entrenamos el modelo por 5 épocas
    model.train()
    for epoch in range(5):  # Incrementa el número de épocas
        for data, target in trainloader:
            optimizer.zero_grad()
            output = model(data)
            loss = loss_fn(output, target)
            loss.backward()
            optimizer.step()

    # Evaluamos el modelo en el conjunto de prueba
    model.eval()
    correct = 0
    with torch.no_grad():
        for data, target in testloader:
            output = model(data)
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    # Calculamos la precisión en el conjunto de prueba y devolverla como el valor objetivo para Optuna
    accuracy = correct / len(testloader.dataset)
    return accuracy

# Ejecutamos el estudio de Optuna para optimizar los hiperparámetros
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=10)  # Número reducido de pruebas por simplicidad

# Imprimimos los mejores hiperparámetros y el mejor valor objetivo encontrado por Optuna
print("Mejores hiperparámetros: {}".format(study.best_params))
print("Mejor valor objetivo: {}".format(study.best_value))

[I 2024-06-13 10:59:31,507] A new study created in memory with name: no-name-cfafb8d7-a084-47e8-807c-3970197faece
[I 2024-06-13 11:00:30,792] Trial 0 finished with value: 0.146 and parameters: {'lr': 0.001104492529386432, 'weight_decay': 0.00038176960887502197, 'momentum': 0.5318415910252537}. Best is trial 0 with value: 0.146.
[I 2024-06-13 11:01:29,590] Trial 1 finished with value: 0.18 and parameters: {'lr': 0.045970324261125885, 'weight_decay': 0.00019255833041217468, 'momentum': 0.2392286428545325}. Best is trial 1 with value: 0.18.
[I 2024-06-13 11:02:28,138] Trial 2 finished with value: 0.109 and parameters: {'lr': 0.008052896813467075, 'weight_decay': 0.00018854240871847832, 'momentum': 0.47287376666745196}. Best is trial 1 with value: 0.18.
[I 2024-06-13 11:03:24,982] Trial 3 finished with value: 0.105 and parameters: {'lr': 0.00031554707295292507, 'weight_decay': 7.732349241300106e-05, 'momentum': 0.9208675257907671}. Best is trial 1 with value: 0.18.
[I 2024-06-13 11:04:25,2

Mejores hiperparámetros: {'lr': 0.03638232759771822, 'weight_decay': 0.0017245746897729779, 'momentum': 0.9429125555636665}
Mejor valor objetivo: 0.378


## 3. Experimentación con modelos secuenciales para NLP:

a. Aplicar Optuna en modelos LSTM o Transformer para tareas como traducción automática o generación de texto. \\
b. Optimizar hiperparámetros como el número de capas, la dimensión de los embeddings y los parámetros específicos de atención. \\
c. Analizar cómo la optimización afecta la calidad del texto generado y la velocidad de convergencia.

In [1]:
!pip install datasets

In [1]:
! pip install -U accelerate
! pip install -U transformers

In [6]:
import datasets
import optuna
import os
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
)

# Cargar dataset
dataset = load_dataset("ade_corpus_v2", "Ade_corpus_v2_classification")
dataset = dataset["train"].train_test_split(0.2)

# Definir el nombre del modelo y tokenizer
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Función de preprocesamiento
def preprocess(examples):
    return tokenizer(
        examples["text"], truncation=True, padding="max_length", max_length=64
    )

# Preprocesar el dataset
dataset = dataset.map(preprocess, batched=True, batch_size=1000)

# Función objetivo para optimización de hiperparámetros
def objective(trial: optuna.Trial):
    model = AutoModelForSequenceClassification.from_pretrained(model_name)
    output_dir = os.path.join("ade-test", f"trial_{trial.number}")  # Directorio de salida único para cada ensayo
    os.makedirs(output_dir, exist_ok=True)

    training_args = TrainingArguments(
        output_dir=output_dir,
        learning_rate=trial.suggest_loguniform("learning_rate", low=4e-5, high=0.01),
        weight_decay=trial.suggest_loguniform("weight_decay", 4e-5, 0.01),
        num_train_epochs=trial.suggest_int("num_train_epochs", low=2, high=5),
        per_device_train_batch_size=16,  # Aumentar tamaño de lote
        per_device_eval_batch_size=16,   # Aumentar tamaño de lote
        evaluation_strategy="epoch",     # Evaluar al final de cada época
        logging_dir=output_dir,         # Guardar logs en el mismo directorio
        logging_steps=100,               # Registrar cada 100 pasos
        disable_tqdm=True,
    )
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=dataset["train"],
        eval_dataset=dataset["test"],
    )
    result = trainer.train()
    return result.training_loss

# Optimizar hiperparámetros
study = optuna.create_study(study_name="hyper-parameter-search", direction="minimize")
study.optimize(func=objective, n_trials=5)  # Reducir el número de trials

print("Mejor valor de pérdida:", study.best_value)
print("Mejores parámetros:", study.best_params)
print("Mejor trial:", study.best_trial)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

Map:   0%|          | 0/4704 [00:00<?, ? examples/s]

[I 2024-06-13 12:06:09,266] A new study created in memory with name: hyper-parameter-search


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-6-ea10b266af07>:37: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate=trial.suggest_loguniform("learning_rate", low=4e-5, high=0.01),
<ipython-input-6-ea10b266af07>:38: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  weight_decay=trial.suggest_loguniform("weight_decay", 4e-5, 0.01),
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: Fu

KeyboardInterrupt: 

## 4. Desarrollo de un sistema de prunning automático:

a. Implementar y configurar el prunning de ensayos en Optuna para detener automáticamente los ensayos menos prometedores y reducir el tiempo de computación. \\
b. Comparar el rendimiento y la eficiencia del proceso de optimización con y sin prunning.

## 5. Integración de técnicas de transfer learning:

a. Experimentar con la optimización de modelos preentrenados en tareas específicas, ajustando hiperparámetros para fine-tuning. \\
b. Evaluar la efectividad de Optuna en la selección de hiperparámetros que maximizan el transfer learning.

En este ejemplo práctico de optimización de hiperparámetros, abordaremos un problema de clasificación binaria.

(basado en http://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html)

Utilizaremos el conjunto de datos Hormigas contra Abejas, que forma parte del conjunto de datos ImageNet. Deberá descargarlo desde aquí: Hormigas contra abejas. Contiene 400 imágenes, ~250 de entrenamiento y ~150 de validación (prueba).

In [1]:
!wget https://download.pytorch.org/tutorial/hymenoptera_data.zip
!unzip hymenoptera_data.zip

--2024-06-13 12:13:00--  https://download.pytorch.org/tutorial/hymenoptera_data.zip
Resolving download.pytorch.org (download.pytorch.org)... 13.33.183.125, 13.33.183.33, 13.33.183.123, ...
Connecting to download.pytorch.org (download.pytorch.org)|13.33.183.125|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 47286322 (45M) [application/zip]
Saving to: ‘hymenoptera_data.zip’

hymenoptera_data.zi 100%[===================>]  45.10M   122MB/s    in 0.4s    

2024-06-13 12:13:00 (122 MB/s) - ‘hymenoptera_data.zip’ saved [47286322/47286322]

Archive:  hymenoptera_data.zip
   creating: hymenoptera_data/
   creating: hymenoptera_data/train/
   creating: hymenoptera_data/train/ants/
  inflating: hymenoptera_data/train/ants/0013035.jpg  
  inflating: hymenoptera_data/train/ants/1030023514_aad5c608f9.jpg  
  inflating: hymenoptera_data/train/ants/1095476100_3906d8afde.jpg  
  inflating: hymenoptera_data/train/ants/1099452230_d1949d3250.jpg  
  inflating: hymenoptera_data/

In [34]:
import torch
import optuna
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
from PIL import Image
from tempfile import TemporaryDirectory
import copy

cudnn.benchmark = True
plt.ion()   # interactive mode

In [35]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = './hymenoptera_data'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

La siguiente función se utilizará para entrenar el modelo:

In [37]:
def train_model(trial, model, criterion, optimizer, num_epochs=4):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

        trial.report(epoch_acc, epoch)
        if trial.should_prune():
            raise optuna.TrialPruned()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    model.load_state_dict(best_model_wts)
    return model, best_acc

Para empezar, es fundamental crear la Función Objetivo. Esta función toma una configuración de hiperparámetros y devuelve su puntuación de evaluación (valor objetivo). Optuna resuelve el problema de la optimización de hiperparámetros al maximizar o minimizar esta Función Objetivo.

La Función Objetivo encapsula el proceso estándar de entrenamiento del modelo. Definimos nuestro modelo, configuramos optimizadores y funciones de pérdida, evaluamos métricas, entre otros pasos. En este ejemplo, evaluaremos la métrica de precisión en el conjunto de validación. También devolveremos su valor desde la Función Objetivo para que Optuna lo utilice en la optimización.

Dentro de la Función Objetivo, debemos definir los hiperparámetros que deseamos optimizar. En Optuna, es posible optimizar diferentes tipos de hiperparámetros, como:

- Números reales (floats).
- Números enteros (integers).
- Categóricos discretos.

En nuestro ejemplo, optimizaremos tres hiperparámetros:

- Red preentrenada. Dado que el conjunto de datos de "Hormigas vs. Abejas" es pequeño, utilizaremos transfer learning para obtener un modelo de buena calidad. Hemos elegido una de las redes entrenadas en ImageNet y reemplazamos las últimas capas completamente conectadas responsables de la clasificación.
- Optimizador: SGD, Adam.
- Tasa de aprendizaje: de 1e-4 a 1e-2.

In [38]:
def objetivo(prueba):

    # Hiperparámetros que queremos optimizar
    params = {
        "nombre_modelo": prueba.suggest_categorical('nombre_modelo', ["resnet18", "alexnet", "vgg16"]),
        "lr": prueba.suggest_loguniform('lr', 1e-4, 1e-2),
        "nombre_optimizador": prueba.suggest_categorical('nombre_optimizador', ["SGD", "Adam"])
    }

    # Obtener el modelo preentrenado
    modelo = get_model(params["nombre_modelo"])
    modelo = modelo.to(device)

    # Definir criterio
    criterio = nn.CrossEntropyLoss()

    # Configurar optimizador
    optimizador = getattr(
        torch.optim, params["nombre_optimizador"]
    )(modelo.parameters(), lr=params["lr"])

    # Entrenar el modelo
    mejor_modelo, mejor_acc = train_model(prueba, modelo, criterio, optimizador, num_epochs=5)

    # Guardar el mejor modelo para cada prueba
    # torch.save(mejor_modelo.state_dict(), f"modelo_prueba_{prueba.number}.pth")

    # Devolver precisión (Valor Objetivo) de la prueba actual
    return mejor_acc


Nota: Para obtener un modelo preentrenado por su nombre, añadiremos una función get_model:

In [36]:
def get_model(model_name: str = "resnet18"):
    if model_name == "resnet18":
        model = models.resnet18(pretrained=True)
        in_features = model.fc.in_features
        model.fc = nn.Linear(in_features, 2)
    elif model_name == "alexnet":
        model = models.alexnet(pretrained=True)
        in_features = model.classifier[1].in_features
        model.classifier = nn.Linear(in_features, 2)
    elif model_name == "vgg16":
        model = models.vgg16(pretrained=True)
        in_features = model.classifier[0].in_features
        model.classifier = nn.Linear(in_features, 2)
    return model

Para empezar a optimizar nuestra Función Objetivo, creamos un nuevo estudio:

In [39]:
# sampler: Queremos usar un muestreador TPE
# pruner: Utilizamos un podador MedianPruner para interrumpir pruebas poco prometedoras
# direction: La dirección de estudio es "maximize" porque queremos maximizar la precisión
# n_trials: Número de pruebas

muestreador = optuna.samplers.TPESampler()
estudio = optuna.create_study(
    sampler=muestreador,
    pruner=optuna.pruners.MedianPruner(
        n_startup_trials=3, n_warmup_steps=5, interval_steps=3
    ),
    direction='maximize')
estudio.optimize(func=objetivo, n_trials=10)


[I 2024-06-13 12:29:09,895] A new study created in memory with name: no-name-2ef921f8-0c24-4cb6-af69-15735010e107
<ipython-input-38-d7cdcd512d58>:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "lr": prueba.suggest_loguniform('lr', 1e-4, 1e-2),
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 122MB/s]


Epoch 0/4
----------
train Loss: 0.9118 Acc: 0.5738
val Loss: 0.5762 Acc: 0.7255

Epoch 1/4
----------
train Loss: 0.6405 Acc: 0.6311
val Loss: 0.5214 Acc: 0.7190

Epoch 2/4
----------
train Loss: 0.6355 Acc: 0.6598
val Loss: 0.5704 Acc: 0.6928

Epoch 3/4
----------
train Loss: 0.5416 Acc: 0.7254
val Loss: 7.6141 Acc: 0.4706

Epoch 4/4
----------
train Loss: 0.6580 Acc: 0.6680


[I 2024-06-13 12:37:19,832] Trial 0 finished with value: 0.7254901960784313 and parameters: {'nombre_modelo': 'resnet18', 'lr': 0.0006535366129131921, 'nombre_optimizador': 'Adam'}. Best is trial 0 with value: 0.7254901960784313.


val Loss: 3.7988 Acc: 0.4837

Training complete in 8m 9s
Best val Acc: 0.725490
Epoch 0/4
----------
train Loss: 0.6474 Acc: 0.6762
val Loss: 0.4841 Acc: 0.7778

Epoch 1/4
----------
train Loss: 0.4968 Acc: 0.7828
val Loss: 0.5511 Acc: 0.6993

Epoch 2/4
----------
train Loss: 0.3929 Acc: 0.8115
val Loss: 0.3859 Acc: 0.8235

Epoch 3/4
----------
train Loss: 0.4035 Acc: 0.8238
val Loss: 0.5735 Acc: 0.7908

Epoch 4/4
----------
train Loss: 0.2916 Acc: 0.8730


[I 2024-06-13 12:40:12,706] Trial 1 finished with value: 0.8235294117647058 and parameters: {'nombre_modelo': 'alexnet', 'lr': 0.00019949812144232608, 'nombre_optimizador': 'Adam'}. Best is trial 1 with value: 0.8235294117647058.


...

val Loss: 0.5663 Acc: 0.7124

Training complete in 2m 25s
Best val Acc: 0.803922


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:09<00:00, 59.8MB/s]


Epoch 0/4
----------


[W 2024-06-13 12:52:37,486] Trial 4 failed with parameters: {'nombre_modelo': 'vgg16', 'lr': 0.007511637588627055, 'nombre_optimizador': 'SGD'} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/optuna/study/_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
  File "<ipython-input-38-d7cdcd512d58>", line 23, in objetivo
    mejor_modelo, mejor_acc = train_model(prueba, modelo, criterio, optimizador, num_epochs=5)
  File "<ipython-input-37-38918a958b01>", line 27, in train_model
    outputs = model(inputs)
  File "/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py", line 1532, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py", line 1541, in _call_impl
    return forward_call(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/torchvision/models/vgg.py", line 66, 

KeyboardInterrupt: 

In [ ]:
print("Mejor prueba: ")
print(estudio.best_trial)

## 6. Análisis de sensibilidad y robustez:

a. Realizar un análisis de sensibilidad para identificar qué hiperparámetros son más influyentes en el rendimiento del modelo. \\
b. Investigar la robustez de los modelos optimizados en condiciones de variación de datos, como ruido o cambios en la distribución de los datos.

In [ ]:
optuna.visualization.plot_parallel_coordinate(estudio)

In [ ]:
optuna.visualization.plot_contour(estudio, params=['optimizer_name','model_name'])

Gráficos de cortes para cada uno de los hiperparámetros:

In [ ]:
optuna.visualization.plot_slice(estudio)

Importancia de los hiperparámetros:

In [ ]:
optuna.visualization.plot_param_importances(estudio)

Trazar el historial de optimización de todos los ensayos de un estudio:

In [ ]:
optuna.visualization.plot_optimization_history(estudio)

Curvas de aprendizaje de los ensayos:

In [ ]:
optuna.visualization.plot_intermediate_values(estudio)

## 7. Automatización y escalabilidad del proceso de optimización (opcional):

a. Desarrollar un framework automatizado que pueda escalar la optimización de hiperparámetros a múltiples máquinas o GPUs. \\
b. Utilizar Optuna en un entorno de computación distribuida para manejar grandes volúmenes de pruebas de hiperparámetros de manera eficiente.

## 8. Documentación de resultados:

a. Preparar una documentación que describa los métodos utilizados, los resultados obtenidos y las recomendaciones para futuras investigaciones. \\
b. Publicar los hallazgos en un artículo de conferencia o revista, enfocándose en cómo la optimización de hiperparámetros puede mejorar significativamente los modelos de aprendizaje profundo (opcional).